**2. Country Code Extraction and Mobile Number Enrichment**
Task:
* Extract the country code from the Address field.
* Enrichment:
        1. Map the extracted country code to its respective international phone dialing code.
        
        2. Prepend this dialing code to the customer's Mobile Number to form a complete, internationally formatted phone number.

In [1]:
server='DESKTOP-HJVSCEN\MSSQLSERVER1'
database='Python ETL'
username='sa'
password='Ka@12345678'

In [2]:
import urllib

ConnectionString = f"""
    DRIVER={{ODBC Driver 18 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;
"""
# URL-encode the connection string for SQLAlchemy
params=urllib.parse.quote_plus(ConnectionString)

In [3]:
from sqlalchemy import create_engine
import pandas as pd

engine=create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

In [4]:
customer_df=pd.read_sql("SELECT * FROM CUSTOMERS",con=engine)
order_df=pd.read_sql("SELECT * FROM ORDERS",con=engine)
transaction_df=pd.read_sql("SELECT * FROM TRANSACTIONS",con=engine)

In [11]:
customer_df['country_code_name']=customer_df['address'].str.slice(-8,-6)
customer_df

,customer_id,customer_name,email,phone,address,registration_date,loyalty_status,loyalty_status_num,country_code_name
0,1,Michelle Kidd,vayala@example.net,6197234258,"USNS Santiago, FPO AE 80872",2025-01-25,Gold,0,AE
1,2,Brad Newton,taylorcatherine@example.net,5376741158,"38783 Oliver Street, West Kristenborough, MT 9...",2023-07-13,Silver,1,MT
2,3,Larry Torres,dsanchez@example.net,8102564505,"6845 Steele Turnpike, West Erikabury, UT 37487",2023-08-18,Bronze,2,UT
3,4,Kimberly Price,jessicaknight@example.com,4232229779,"1631 Alexis Meadows, Lake Amanda, CA 75179",2024-12-08,Gold,0,CA
4,5,Matthew Phillips,qwilliams@example.com,2207633522,"2274 Williams Heights Suite 895, Andersonhaven...",2024-02-03,Gold,0,OR
...,...,...,...,...,...,...,...,...,...
995,996,Jerry Mcdaniel,walkerlisa@example.net,6389899441,"34746 Smith Gateway, New Sarah, AS 12715",2025-02-10,Silver,1,AS
996,997,Jodi Simpson,eric24@example.org,4836252940,"2876 Tucker Road Suite 947, North Tommyborough...",2024-04-18,Bronze,2,AL
997,998,Crystal Brown,pshaffer@example.net,3907473088,"095 Janice Forest Suite 570, Boltonmouth, DE 7...",2024-08-30,Bronze,2,DE
998,999,Gregory Duarte,caitlindunlap@example.org,2574098196,"Unit 6377 Box 7662, DPO AP 03300",2024-05-16,Gold,0,AP


In [12]:
country=pd.read_csv('../Datasets/countrycodes.csv')

In [18]:
customer_df=pd.merge(customer_df,country,left_on='country_code_name',right_on='ISO2',how='left')

In [20]:
customer_df['CallingCode'].fillna("1",inplace=True)

C:\Users\Tanuj\AppData\Local\Temp\ipykernel_18924\145036996.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  customer_df['CallingCode'].fillna("1",inplace=True)


In [22]:
customer_df['phone']='+'+customer_df['CallingCode']+' '+customer_df['phone']

In [23]:
customer_df

,customer_id,customer_name,email,phone,address,registration_date,loyalty_status,loyalty_status_num,country_code_name,Country,ISO2,CallingCode
0,1,Michelle Kidd,vayala@example.net,+971 6197234258,"USNS Santiago, FPO AE 80872",2025-01-25,Gold,0,AE,United Arab Emirates,AE,971
1,2,Brad Newton,taylorcatherine@example.net,+356 5376741158,"38783 Oliver Street, West Kristenborough, MT 9...",2023-07-13,Silver,1,MT,Malta,MT,356
2,3,Larry Torres,dsanchez@example.net,+1 8102564505,"6845 Steele Turnpike, West Erikabury, UT 37487",2023-08-18,Bronze,2,UT,NaN,NaN,1
3,4,Kimberly Price,jessicaknight@example.com,+1 4232229779,"1631 Alexis Meadows, Lake Amanda, CA 75179",2024-12-08,Gold,0,CA,Canada,CA,1
4,5,Matthew Phillips,qwilliams@example.com,+1 2207633522,"2274 Williams Heights Suite 895, Andersonhaven...",2024-02-03,Gold,0,OR,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Jerry Mcdaniel,walkerlisa@example.net,+1 6389899441,"34746 Smith Gateway, New Sarah, AS 12715",2025-02-10,Silver,1,AS,NaN,NaN,1
996,997,Jodi Simpson,eric24@example.org,+355 4836252940,"2876 Tucker Road Suite 947, North Tommyborough...",2024-04-18,Bronze,2,AL,Albania,AL,355
997,998,Crystal Brown,pshaffer@example.net,+49 3907473088,"095 Janice Forest Suite 570, Boltonmouth, DE 7...",2024-08-30,Bronze,2,DE,Germany,DE,49
998,999,Gregory Duarte,caitlindunlap@example.org,+1 2574098196,"Unit 6377 Box 7662, DPO AP 03300",2024-05-16,Gold,0,AP,NaN,NaN,1


In [24]:
customer_df.drop(columns=['ISO2','CallingCode'])

,customer_id,customer_name,email,phone,address,registration_date,loyalty_status,loyalty_status_num,country_code_name,Country
0,1,Michelle Kidd,vayala@example.net,+971 6197234258,"USNS Santiago, FPO AE 80872",2025-01-25,Gold,0,AE,United Arab Emirates
1,2,Brad Newton,taylorcatherine@example.net,+356 5376741158,"38783 Oliver Street, West Kristenborough, MT 9...",2023-07-13,Silver,1,MT,Malta
2,3,Larry Torres,dsanchez@example.net,+1 8102564505,"6845 Steele Turnpike, West Erikabury, UT 37487",2023-08-18,Bronze,2,UT,NaN
3,4,Kimberly Price,jessicaknight@example.com,+1 4232229779,"1631 Alexis Meadows, Lake Amanda, CA 75179",2024-12-08,Gold,0,CA,Canada
4,5,Matthew Phillips,qwilliams@example.com,+1 2207633522,"2274 Williams Heights Suite 895, Andersonhaven...",2024-02-03,Gold,0,OR,NaN
...,...,...,...,...,...,...,...,...,...,...
995,996,Jerry Mcdaniel,walkerlisa@example.net,+1 6389899441,"34746 Smith Gateway, New Sarah, AS 12715",2025-02-10,Silver,1,AS,NaN
996,997,Jodi Simpson,eric24@example.org,+355 4836252940,"2876 Tucker Road Suite 947, North Tommyborough...",2024-04-18,Bronze,2,AL,Albania
997,998,Crystal Brown,pshaffer@example.net,+49 3907473088,"095 Janice Forest Suite 570, Boltonmouth, DE 7...",2024-08-30,Bronze,2,DE,Germany
998,999,Gregory Duarte,caitlindunlap@example.org,+1 2574098196,"Unit 6377 Box 7662, DPO AP 03300",2024-05-16,Gold,0,AP,NaN


In [25]:
customer_df.to_sql("customers", con=engine,index=False,if_exists='replace')

130